In [216]:
import sys

sys.path.append("..")
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from dory.linear.ols import OrdinaryLeastSquaresRegressor
from dory.scaler import NonCombinatoryExplosionPolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

In [217]:
california_df = pd.read_csv("california.csv", header=None)
california_df.describe()

,0,1,2,3,4,5,6,7,8
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3.948099,28.733000,5.417844,1.087299,1423.847000,2.897038,35.765440,-119.708800,2.111088
std,1.873914,12.961239,1.875375,0.285668,1396.022801,0.763631,2.197077,2.023945,1.142300
min,0.706800,2.000000,1.411290,0.526316,9.000000,1.345506,32.550000,-124.180000,0.325000
25%,2.674075,18.000000,4.458198,1.006177,774.500000,2.408481,33.920000,-121.880000,1.242500
50%,3.667450,29.000000,5.281518,1.046735,1169.500000,2.794112,34.420000,-118.905000,1.850500
75%,4.788375,38.000000,6.031825,1.097236,1681.000000,3.248397,37.760000,-118.040000,2.695500
max,15.000000,52.000000,26.465969,5.424084,28566.000000,9.196203,41.860000,-115.490000,5.000010


In [218]:

X = california_df.iloc[:, :-1].to_numpy()
y = california_df.iloc[:, -1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42069)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 8) (200, 8) (800,) (200,)


In [219]:
features_names = california_df.columns[:-1]
target_name = california_df.columns[-1]
feature_scaler = MinMaxScaler()
target_scaler = StandardScaler()

X_train = feature_scaler.fit_transform(X_train)
y_train = target_scaler.fit_transform(y_train.reshape(-1, 1))
pd.DataFrame(y_train).describe()

,0
count,8.000000e+02
mean,-3.153033e-16
std,1.000626e+00
min,-1.548903e+00
25%,-7.579848e-01
50%,-2.537401e-01
75%,5.031620e-01
max,2.451690e+00


In [220]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,0.228198,0.531125,0.173483,0.159820,0.049425,0.196727,0.344650,0.513140
std,0.133920,0.258254,0.080486,0.077415,0.050736,0.096423,0.235019,0.232825
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.139173,0.300000,0.129117,0.136414,0.027034,0.134698,0.145048,0.263521
50%,0.207473,0.520000,0.169550,0.147727,0.040848,0.182651,0.203983,0.594937
75%,0.288854,0.720000,0.202630,0.162661,0.057867,0.239631,0.559742,0.707710
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [221]:
pipe = Pipeline(
    [
        ("poly", NonCombinatoryExplosionPolynomialFeatures()),
        ("regressor", OrdinaryLeastSquaresRegressor()),
    ]
)
pipe

,steps,"[('poly', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,degree,2
,fit_intercept,True
,l2_alpha,0.0


In [222]:
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid={
        "poly__degree": range(1, 13 + 1),
        "regressor__l2_alpha": [0.0, 0.1, 1.0, 100],
    },
    verbose=3,
)

In [223]:
grid_search.fit(X_train, y_train)


/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 1.0
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 1.0
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regu

Fitting 5 folds for each of 52 candidates, totalling 260 fits
[CV 1/5] END poly__degree=1, regressor__l2_alpha=0.0;, score=0.659 total time=   0.0s
[CV 2/5] END poly__degree=1, regressor__l2_alpha=0.0;, score=0.677 total time=   0.0s
[CV 3/5] END poly__degree=1, regressor__l2_alpha=0.0;, score=0.727 total time=   0.0s
[CV 4/5] END poly__degree=1, regressor__l2_alpha=0.0;, score=0.598 total time=   0.0s
[CV 5/5] END poly__degree=1, regressor__l2_alpha=0.0;, score=0.663 total time=   0.0s
[CV 1/5] END poly__degree=1, regressor__l2_alpha=0.1;, score=0.659 total time=   0.0s
[CV 2/5] END poly__degree=1, regressor__l2_alpha=0.1;, score=0.676 total time=   0.0s
[CV 3/5] END poly__degree=1, regressor__l2_alpha=0.1;, score=0.726 total time=   0.0s
[CV 4/5] END poly__degree=1, regressor__l2_alpha=0.1;, score=0.594 total time=   0.0s
[CV 5/5] END poly__degree=1, regressor__l2_alpha=0.1;, score=0.673 total time=   0.0s
[CV 1/5] END poly__degree=1, regressor__l2_alpha=1.0;, score=0.648 total time=

/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 100
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 100
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regu

[CV 1/5] END poly__degree=4, regressor__l2_alpha=100;, score=0.206 total time=   0.0s
[CV 2/5] END poly__degree=4, regressor__l2_alpha=100;, score=0.193 total time=   0.0s
[CV 3/5] END poly__degree=4, regressor__l2_alpha=100;, score=0.225 total time=   0.0s
[CV 4/5] END poly__degree=4, regressor__l2_alpha=100;, score=0.164 total time=   0.0s
[CV 5/5] END poly__degree=4, regressor__l2_alpha=100;, score=0.231 total time=   0.0s
[CV 1/5] END poly__degree=5, regressor__l2_alpha=0.0;, score=-36531.427 total time=   0.0s
[CV 2/5] END poly__degree=5, regressor__l2_alpha=0.0;, score=-20.732 total time=   0.0s
[CV 3/5] END poly__degree=5, regressor__l2_alpha=0.0;, score=-17.210 total time=   0.0s
[CV 4/5] END poly__degree=5, regressor__l2_alpha=0.0;, score=0.596 total time=   0.0s
[CV 5/5] END poly__degree=5, regressor__l2_alpha=0.0;, score=0.647 total time=   0.0s
[CV 1/5] END poly__degree=5, regressor__l2_alpha=0.1;, score=0.622 total time=   0.0s
[CV 2/5] END poly__degree=5, regressor__l2_al

...

/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 1.0
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 100
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regu

[CV 4/5] END poly__degree=13, regressor__l2_alpha=1.0;, score=0.577 total time=   0.0s
[CV 5/5] END poly__degree=13, regressor__l2_alpha=1.0;, score=0.682 total time=   0.0s
[CV 1/5] END poly__degree=13, regressor__l2_alpha=100;, score=0.237 total time=   0.0s


/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 100
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regularization with alpha 100
/home/flemis/codes/dory/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dory.linear.ols:Using L2 regu

[CV 2/5] END poly__degree=13, regressor__l2_alpha=100;, score=0.208 total time=   0.0s
[CV 3/5] END poly__degree=13, regressor__l2_alpha=100;, score=0.234 total time=   0.0s
[CV 4/5] END poly__degree=13, regressor__l2_alpha=100;, score=0.170 total time=   0.0s
[CV 5/5] END poly__degree=13, regressor__l2_alpha=100;, score=0.252 total time=   0.0s


,estimator,Pipeline(step...Regressor())])
,param_grid,"{'poly__degree': range(1, 14), 'regressor__l2_alpha': [0.0, 0.1, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,None
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,3


In [224]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_
pipe = grid_search.best_estimator_
print("Best params:", best_params)
print("Best MSE score:", best_score)

Best params: {'poly__degree': 3, 'regressor__l2_alpha': 0.1}
Best MSE score: 0.6756698725820766
